In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from dateutil.parser import parse

In [2]:
# cols = ["MachineName","SourceName","EventTime", "Message","Quality","Condition","Attributes"]
df = pd.read_excel("/home/waris/Github/tupras-analysis/new-alarm/proses_alarm.xls",skipinitialspace=True)

In [3]:

# initial processing
for i in range(0, len(df["EventTime"])):
    d = df["EventTime"][i]
    d = d.replace(".000000000","")
    d = d.replace("/","-")
    df["EventTime"][i] = parse(d)
    s = df["Message"][i]
    df["Message"][i] = " ".join(s.split())


<ipython-input-3-a5c482d15d62>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["EventTime"][i] = parse(d)
<ipython-input-3-a5c482d15d62>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Message"][i] = " ".join(s.split())


In [4]:
print("Date-Type:", type(df["EventTime"][1]))
print(df)

Date-Type: <class 'datetime.datetime'>
      MachineName            ServerProgID ServerNodeName  \
0          PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
1          PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
2          PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
3          PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
4          PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
...           ...                     ...            ...   
42089      PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
42090      PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
42091      PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
42092      PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   
42093      PHD47B  Yokogawa.ExaopcAECS1.1        STN0830   

             SubscriptionName SourceName            EventTime  EventTime_MS  \
0      Yokogawa.ExaopcAECS1.1  48TIC2026  2020-01-27 09:55:34    -980093184   
1      Yokogawa.ExaopcAECS1.1  48TIC2026  2020-01-27 09:55:36    -960093184   
2  

In [5]:
# client = MongoClient()
# db = client["tupras-db"]
# collection_raw_alarm = db["alarm"]
# data = json.loads(df.to_json(orient='records'))
# collection_raw_alarm.remove()
# print(data)
# collection_raw_alarm.insert(data)

# Preprocessing of Alarm data

There are different types of alarm data but Excel file only contains "Process Alarm Data".

**Excel Columns Description**
* MachineName: It represent the unit (e.g.,Plant-47)
* SourceName: The names of the devices operating in a unit (e.g., controller, transmitter, and calculator block names)
    1. Five types of controllers: TIC, PIC, FIC, LIC, and AIC.
    2. Three types of transmitters: TI, FI, LI
    3. Calculator block names ?

* EventTime: The date when an alarm is appeared. It also represent when it recovered right?

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state. 

**Questions:**
1. Before alarm data analysis, if ALM, NR and RECOVER messages are seen, these rows must be deleted. Why?
2. Alarm-off 
3. shelving of alarms
4. Can alarm cannot trigger twice? 
5. 

In [6]:
sources = {}
source_conditions = {}
conditions = {} # Vel, IOP ALM, NR etc
key = ""
conditions["Recover"] = 0
for i in range(0, len(df["EventTime"])):
    key = df["SourceName"][i]
    temp_dict = {"Quality":df["Quality"][i], "Mask":df["Mask"][i], "NewState":df["NewState"][i], "Status": df["Status"][i] ,"SourceName": key,"EventTime": df["EventTime"][i], "Message":df["Message"][i],"Condition": df["Condition"][i],"Attributes":df["Attributes"][i]}
    
    if sources.get(key) != None:
        sources[key].append(temp_dict)
        
        #source conditions
        d =  source_conditions[key]
        if d.get(temp_dict["Condition"]) == None:
            d[temp_dict["Condition"]] = 0    
        d[temp_dict["Condition"]]  += 1
        d["Total"] += 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        # conditions           
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] += 1
        if temp_dict["Message"].strip().find("NR") != -1:
            print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1
    else:
        sources[key] = []
        source_conditions[key] = {}
        sources[key].append(temp_dict)
        d = source_conditions[key]
        d[temp_dict["Condition"]] = 1
        d["Total"] = 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] +=1
        if temp_dict["Message"].strip().find("NR") != -1:
            print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1


# sorting the source conditions dictionary based on the number of alarms related to each sensor
source_conditions = {k:v for k,v in sorted(source_conditions.items(), key=lambda item: item[1]["Total"],reverse=True)}
conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
48XL002-ANN 48G-1B STATUS NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47FD-032H-AN C-109 MERDIV ALEV YUK NR
47FD-033F-AN E-113 ALEV ARIZA NR
47FD-032HH-A C-109 MERDIV ALEV YUKYUK NR
47FD-032H-AN C-109 MERDIV ALEV YUK NR
47FD-032HH-A C-109 MERDIV ALEV YUKYUK NR
47UAL801-ANN K-2 C MEKANIK LUBRIKATOR NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47FD-032H-AN C-109 M

In [7]:
# Total number of disctinct SourceNames in the excel sheet.

print("Total Number of Sourcenames: {s}\n\nSources List \n\n {l}".format(s=len(sources.keys()), l = sources.keys()))

Total Number of Sourcenames: 109

Sources List 

 dict_keys(['48TIC2026', '47TI931A', '47TAL003AB-ANN', '47FLI-032', '47TI002C', '47AI1503', '47TI003A', '47PIC1755', '47PDI1746', '47TI1507', '47TIC1511', '47TI3519', '47TIC3520', '47TI003B', '47XL015-ANN', '47XL016-ANN', '47LI003', '47TI2076', '48XL002-ANN', '47FSL019-ANN', '47TI1769', '47TI1506', '47LIC1509', 'SIO-18XA001', '47FI1005', '47TI2086', '47FIC021', '47TI3102', '47AIT2011', '47FLI-042', '47TI1512B', '47TI1512A', '47FD-032H-ANN', '47FD-032HH-ANN', '47FLI-039', '47FD-033F-ANN', '47TI1713', '47UAL801-ANN', '48XL006-ANN', '47PDI003', '47FLI-022', '47FIC1524_CALX', '48PAL013-ANN', '48XL004-ANN', '47LIC005', '47FD-036H-ANN', '47FD-036HH-ANN', '47FLI-036', '47LIC1501', '47FIC1563', '47TI2003', '47FIC037', '47TI1512ABC', '47TI1529', '47LIC1555', '47TI1512C', '47FI1568', '47TI2083', '47TI414O', '47TI416', '48LIC009', '47FIC3102', '47PIC1505', '48PI2025', '47FIC1522A', '48FIX008', '48AIC001', '47FLI-041', '47FLI-037', '47LIC3408', '48H

In [8]:
# How many times each SourceName has been triggered (activation + Deactivation)?
total  = 0
i =0
for key in source_conditions.keys():
    s1 = "Source: " + key + ", Total: {} | ".format(source_conditions[key]["Total"])
    s2 = ""
    total += source_conditions[key]["Total"]
    for k in source_conditions[key].keys():
        if k == "Total" or k == "Recover":
            continue
        s2 = s2 + k +" "+ str(source_conditions[key][k]) + ", "    
    s1 = s1 + s2
    print("{}. ".format(i),s1)
    i +=1
    if i == 10:
        break
print("\n\n Total Number of Entries in the excel:", total)
# assert(total== len(df["SourceName"]))

0.  Source: 48TIC2026, Total: 25220 | VEL- 11675, VEL+ 13545, 
1.  Source: 47TI931A, Total: 10980 | IOP 10980, 
2.  Source: 47FLI-032, Total: 2395 | CNF 1176, IOP 1217, IOP- 2, 
3.  Source: 47AI1503, Total: 1460 | LO 1460, 
4.  Source: 47TI002C, Total: 668 | LLL 668, 
5.  Source: 47PDI1746, Total: 264 | IOP 264, 
6.  Source: 47FSL019-ANN, Total: 181 | ALM 181, 
7.  Source: 47TI003A, Total: 159 | LTRP 159, 
8.  Source: 47TI003B, Total: 84 | LTRP 84, 
9.  Source: 47TAL003AB-ANN, Total: 65 | ALM 65, 


 Total Number of Entries in the excel: 41476


In [9]:
i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
    if i == 10:
        break

0. Source: 48TIC2026, # of activations= 12605, # of deactivations = 12615, Diff = -10
1. Source: 47TI931A, # of activations= 5492, # of deactivations = 5488, Diff = 4
2. Source: 47FLI-032, # of activations= 1250, # of deactivations = 1145, Diff = 105
3. Source: 47AI1503, # of activations= 740, # of deactivations = 720, Diff = 20
4. Source: 47TI002C, # of activations= 336, # of deactivations = 332, Diff = 4
5. Source: 47PDI1746, # of activations= 133, # of deactivations = 131, Diff = 2
6. Source: 47FSL019-ANN, # of activations= 98, # of deactivations = 83, Diff = 15
7. Source: 47TI003A, # of activations= 84, # of deactivations = 75, Diff = 9
8. Source: 47TI003B, # of activations= 43, # of deactivations = 41, Diff = 2
9. Source: 47TAL003AB-ANN, # of activations= 33, # of deactivations = 32, Diff = 1


In [10]:
# number of times each condition appear
for k in conditions.keys():
    if k == "Recover":
        continue
    print(k,conditions[k])

VEL+ 13545
IOP 12574
VEL- 11676
LO 1499
CNF 1220
LLL 672
ALM 382
LTRP 257
HI 117
HHH 60
ANS- 30
HTRP 21
HH 19
DV+ 11
LL 4
IOP- 4
DV- 3


In [11]:
print("Recover", conditions["Recover"])

Recover 20894


### Which message is related to which message (i.e., activation and deactivation) ?
Dispalying top 10 **SourceName** with 30 alarms in sorted order by **EventTime**

In [12]:
# conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

i = 0
for sname in source_conditions.keys():
    
    print("=======Source {}: {}========".format(i,sname))
    single_source_alarms = sources[sname]
    single_source_alarms = [v for v in sorted(single_source_alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    
    j = 0
    for alarm in single_source_alarms:
        print("{},---,{},--- Message: {}".format(alarm["EventTime"],alarm["Condition"], alarm["Message"]))
        j += 1
#         if j == 30:
#             break
    i += 1
    if i == 10:
        break


=======Source 0: 48TIC2026========
2020-01-27 09:36:35,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 09:36:38,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+ Recover
2020-01-27 09:36:45,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL- Recover
2020-01-27 09:36:45,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+
2020-01-27 09:36:47,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL+ Recover
2020-01-27 09:36:47,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL-
2020-01-27 09:36:48,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 09:36:48,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL+
2020-01-27 09:36:52,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 09:36:52,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL+
2020-01-27 09:36:55,---,VEL-,--- Me

2020-01-27 10:09:57,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL+ Recover
2020-01-27 10:09:57,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 10:09:59,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL- Recover
2020-01-27 10:09:59,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+
2020-01-27 10:10:00,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+ Recover
2020-01-27 10:10:01,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 10:10:02,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 10:10:05,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 10:10:07,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL- Recover
2020-01-27 10:10:07,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+
2020-01-27 10:10:09,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDE

2020-01-27 10:34:46,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL- Recover
2020-01-27 10:34:46,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+
2020-01-27 10:34:48,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.5 C VEL+ Recover
2020-01-27 10:34:48,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.5 C VEL-
2020-01-27 10:34:49,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL- Recover
2020-01-27 10:34:51,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.5 C VEL-
2020-01-27 10:34:52,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 10:35:01,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL-
2020-01-27 10:35:03,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.0 C VEL- Recover
2020-01-27 10:35:05,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.7 C VEL-
2020-01-27 10:35:06,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDE

2020-01-27 10:46:17,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL+ Recover
2020-01-27 10:46:17,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 10:46:18,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL- Recover
2020-01-27 10:46:19,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.1 C VEL+
2020-01-27 10:46:21,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL+ Recover
2020-01-27 10:46:26,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL-
2020-01-27 10:46:27,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 10:46:27,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL+
2020-01-27 10:46:29,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.6 C VEL+ Recover
2020-01-27 10:46:29,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.6 C VEL-
2020-01-27 10:46:31,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDE

2020-01-27 10:53:35,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.2 C VEL- Recover
2020-01-27 10:53:35,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.2 C VEL+
2020-01-27 10:53:37,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL+ Recover
2020-01-27 10:53:47,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL-
2020-01-27 10:53:49,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.0 C VEL- Recover
2020-01-27 10:53:49,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.0 C VEL+
2020-01-27 10:53:51,---,VEL+,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.6 C VEL+ Recover
2020-01-27 10:53:51,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.6 C VEL-
2020-01-27 10:53:52,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.3 C VEL- Recover
2020-01-27 10:53:54,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDEA PV = -4.4 C VEL-
2020-01-27 10:53:56,---,VEL-,--- Message: 48TIC2026 E208 CKS TEMIZ MDE

...

2020-01-27 12:46:41,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 12:46:49,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 12:46:51,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 12:46:53,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 12:46:55,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 12:47:01,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 12:47:07,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 12:47:08,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 12:47:12,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 12:47:14,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 12:47:16,---,IOP,--- Message: 47TI931A C5 1.YTK G

2020-01-27 14:11:03,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:11:05,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:11:16,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:11:21,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:11:23,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:11:26,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:11:27,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:11:30,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:11:39,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:11:43,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:11:46,---,IOP,--- Message: 47TI931A C5 1.YTK G

2020-01-27 14:39:03,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:39:04,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:39:06,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:39:08,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:39:10,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:39:13,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:39:15,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:39:21,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:39:24,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:39:25,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:39:27,---,IOP,--- Message: 47TI931A C5 1.YTK G

2020-01-27 14:50:19,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:50:21,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:50:28,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:50:32,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:50:37,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:50:39,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:50:41,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:50:43,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:50:55,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2020-01-27 14:50:58,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2020-01-27 14:51:00,---,IOP,--- Message: 47TI931A C5 1.YTK G

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



**Find the duration of activation of each Alarm i.e., duration =  DeactivationTime - ActivationTime**

Compute mean, median, mode, on the duration...

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state.


In [15]:
from datetime import timedelta
import statistics

def activationAndDeactivation(alarms):
    """Returns list of dict  and wits stats"""
    queue = []
    stats = {} 
    start_end = []
    
    alarms = [v for v in sorted(alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    for alarm in alarms:
#         print(alarm["Message"])
        if alarm["Message"].find("Recover") == -1 and alarm["Message"].find("NR") == -1:
            queue.append(alarm) # enqueue the activation
        else:
            if len(queue)== 0:
#                 print("(", alarm["SourceName"],alarm["Message"], ")")
                continue
            d = queue.pop(0) # dequeue
            d = {k:v for k,v in d.items()}
            d["StartTime"] = d["EventTime"]
            d["EndTime"] = alarm["EventTime"]
            d["EndMessage"] = alarm["Message"]
            del d["EventTime"]
            
            start_end.append(d)
#     print("Remaining Elements in Queue", len(queue))
    temps = []
    for alarm in start_end:
        temps.append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))

    stats["timedeltas"] = [v for v in sorted(temps)]
    stats["mean"] =  round(statistics.mean(stats["timedeltas"]),3) 
    stats["median"] = statistics.median(stats["timedeltas"])
    stats["mode"] = statistics.mode(stats["timedeltas"])
    stats["min"] = stats["timedeltas"][0]
    stats["max"] = stats["timedeltas"][-1]
    return (start_end, stats)

In [39]:

i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
    if i > 4:
        break

print ("==================================================================================")
i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
    if i > 10: 
        break
      
    alarms = sources[sname]
    _, stats = activationAndDeactivation(alarms)
    print("{}. source:{}, Total alarms:{} | min = {}, max = {}, median = {}, mode = {}, mean = {}".format(i,sname,len(stats["timedeltas"]) ,stats["min"], stats["max"], stats["median"], stats["mode"], stats["mean"]))
    i += 1
print("Note: All the stats are in secods")

0. Source: 48TIC2026, # of activations= 12605, # of deactivations = 12615, Diff = -10
1. Source: 47TI931A, # of activations= 5492, # of deactivations = 5488, Diff = 4
2. Source: 47FLI-032, # of activations= 1250, # of deactivations = 1145, Diff = 105
3. Source: 47AI1503, # of activations= 740, # of deactivations = 720, Diff = 20
4. Source: 47TI002C, # of activations= 336, # of deactivations = 332, Diff = 4
1. source:48TIC2026, Total alarms:12605 | min = 1.0, max = 30571.0, median = 14.0, mode = 13.0, mean = 25.273
2. source:47TI931A, Total alarms:5485 | min = 0.0, max = 167205.0, median = 6.0, mode = 1.0, mean = 107.084
3. source:47FLI-032, Total alarms:1142 | min = 7.0, max = 168081.0, median = 47.0, mode = 10.0, mean = 270.074
4. source:47AI1503, Total alarms:720 | min = 4.0, max = 8903.0, median = 519.0, mode = 365.0, mean = 652.722
5. source:47TI002C, Total alarms:331 | min = 0.0, max = 1745.0, median = 5.0, mode = 4.0, mean = 36.544
6. source:47PDI1746, Total alarms:131 | min = 1.

In [38]:
# exporintg data to csv
csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta_Target(sec)": []} 

i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
    if i > 10: 
        break
    i += 1    
#     sname = "48TIC2026"

    alarms = sources[sname]
    alarms,stats = activationAndDeactivation(alarms)

    for alarm in alarms:
        csv["SourceName"].append(alarm["SourceName"]) 
        csv["StartTime"].append(alarm["StartTime"]) 
        csv["EndTime"].append(alarm["EndTime"])
        csv["TimeDelta_Target(sec)"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
        csv["Message"].append(alarm["Message"])
        csv["RecoveryMessage"].append(alarm["EndMessage"])
        csv["Quality"].append(alarm["Quality"])
        csv["Condition"].append(alarm["Condition"])
        csv["Mask"].append(alarm["Mask"])
        csv["NewState"].append(alarm["NewState"])
        csv["Status"].append(alarm["Status"])


print("Total Entries :", len(csv["Status"]))
df = pd.DataFrame(csv)
df.to_csv("stats.csv", index = False)


Total Entries : 20639
